# Logging with App Insights

See [HERE](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-log-pipelines-application-insights)

In [2]:
import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep='\t')

compute_target = 'low-prio-compute'

Azure ML SDK Version:  1.26.0
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.
luisdel-ml	westeurope	luisdel


In [3]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep

# Connecting to the workspace and compute target not shown

# Add pip dependency on OpenCensus
dependencies = CondaDependencies()
dependencies.add_pip_package("opencensus-ext-azure>=1.0.1")
run_config = RunConfiguration(conda_dependencies=dependencies)

# Add environment variable with Application Insights Connection String
# Replace the value with your own connection string
run_config.environment.environment_variables = {
    "APPLICATIONINSIGHTS_CONNECTION_STRING": 'xxx'
}

# Configure step with runconfig
sample_step = PythonScriptStep(
        script_name="sample_step.py",
        compute_target=compute_target,
        runconfig=run_config
)

# Submit new pipeline run
pipeline = Pipeline(workspace=ws, steps=[sample_step])
pipeline.submit(experiment_name="Logging_Experiment")

Created step sample_step.py [c33bf9ac][8953bb80-5313-497d-b8a5-ffbee81d6fcf], (This step will run and generate new outputs)
Submitted PipelineRun 04bdf307-1f3d-4963-b917-bc36ab95a0e4
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/04bdf307-1f3d-4963-b917-bc36ab95a0e4?wsid=/subscriptions/0d42ef99-c937-43f7-8ff1-1e0fd4e5a4b7/resourcegroups/luisdel/workspaces/luisdel-ml&tid=1c27a794-233b-4135-91e9-067074be85fd


Experiment,Id,Type,Status,Details Page,Docs Page
Logging_Experiment,04bdf307-1f3d-4963-b917-bc36ab95a0e4,azureml.PipelineRun,Preparing,Link to Azure Machine Learning studio,Link to Documentation


# Logging with Custom Dimensions

By default, logs forwarded to Application Insights won't have enough context to trace back to the run or experiment. To make the logs actionable for diagnosing issues, additional fields are needed.

See step sample script for a sample on how to do this